In [1]:
import numpy as np
import matplotlib.pyplot as plt
from bigfile import BigFile


In [ ]:
snap = 599
file = "/hildafs/datasets/Asterix/BH_details_bigfile2/BH-Details-R%03d"%snap
bf = BigFile(file)
N = 100000
ids = bf["BHID"][:]
reshift = bf["z"][:]


In [ ]:
idsset = np.unique(ids)
print("Number of unique ids: ", len(idsset))

zset = np.unique(reshift)

In [ ]:
plt.hist(redshift, bins = 20)
plt.show()

In [2]:
snap = 577
file = "/hildafs/datasets/Asterix/BH_details_bigfile2/BH-Details-R%03d"%snap
bf = BigFile(file)
N = 100000
gasvel = bf["GasVel"][:]
# reshift = bf["z"][:]
